In [1]:
dataset_path = r'G:\RAVISIR\dataset1'

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 27 16:04:09 2018

@author: Ravi
"""
 
import sys 
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
from keras.applications.mobilenet import MobileNet
# fix seed for reproducible results (only works on CPU, not GPU)
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)

# hyper parameters for model
nb_classes = 2  # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
#img_width, img_height = 299, 299  # change based on the shape/structure of your images for inception etc
img_width, img_height = 224, 224  # change based on the shape/structure of your images for squeezenet

batch_size = 32  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 10  # number of iteration the algorithm gets trained.
learn_rate = 1e-4  # sgd learning rate
momentum = .9  # sgd momentum to avoid local minimum
transformation_ratio = .05  # how aggressive will be the data augmentation/transformation
nb_train_samples = 240  # Total number of train samples. NOT including augmented images
nb_validation_samples = 96  # Total number of train samples. NOT including augmented images.


def train(train_data_dir, validation_data_dir, model_path):
    # Pre-Trained CNN Model using imagenet dataset for pre-trained weights
    # base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
    
    if K.image_data_format() == 'channels_first': 
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)
    
    base_model = MobileNet(input_shape=input_shape, weights='imagenet', include_top=False)

    # # Top Model Block
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    # predictions = Dense(nb_classes, activation='softmax')(x)
    
    # add a global spatial average pooling layer
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    # # let's add a fully-connected layer
    # x = Dense(1024, activation='relu', name='fc1')(x)
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(nb_classes, activation='softmax', name='predictions')(x)

    # add your top layer block to your base model
    model = Model(base_model.input, predictions)
    print(model.summary())

    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all layers of the based model that is already pre-trained.
    for layer in base_model.layers:
        layer.trainable = False

    # Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
    # To save augmentations un-comment save lines and add to your flow parameters.
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1. / 255)

    # os.makedirs(os.path.join(os.path.abspath(train_data_dir), 'preview'), exist_ok=True)
    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=[img_width, img_height],
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
    # save_to_dir=os.path.join(os.path.abspath(train_data_dir), '../preview')
    # save_prefix='aug',
    # save_format='jpeg')
    # use the above 3 commented lines if you want to save and look at how the data augmentations look like

    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=[img_width, img_height],
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')

    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

    # save weights of best training epoch: monitor either val_loss or val_acc

    top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_acc', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-train-top-layer', histogram_freq=0, write_graph=False, write_images=False)
    ]

    # Train Simple CNN
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch / 5,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=callbacks_list)

    # verbose
    print("\nStarting to Fine Tune Model\n")

    # add the best weights from the train top model
    # at this point we have the pre-train weights of the base model and the trained weight of the new/added top model
    # we re-load model weights to ensure the best epoch is selected and not the last one.
    model.load_weights(top_weights_path)

    # based_model_last_block_layer_number points to the layer in your model you want to train.
    # For example if you want to train the last block of a 19 layer VGG16 model this should be 15
    # If you want to train the last Two blocks of an Inception model it should be 172
    # layers before this number will used the pre-trained weights, layers above and including this number
    # will be re-trained based on the new data.
    for layer in model.layers[:based_model_last_block_layer_number]:
        layer.trainable = False
    for layer in model.layers[based_model_last_block_layer_number:]:
        layer.trainable = True

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
                  
    
    # save weights of best training epoch: monitor either val_loss or val_acc
    final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        #EarlyStopping(monitor='val_loss', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-fine-tune', histogram_freq=0, write_graph=False, write_images=False)
    ]
    
 


    # fine-tune the model
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=callbacks_list)

    # save model
    model_json = model.to_json()
    with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
        json_file.write(model_json)


train(r'G:\RAVISIR\dataset1\train', r'G:\RAVISIR\dataset1\validation', '.')

# if __name__ == '__main__':
#     if not len(sys.argv) == 3:
#         print('Arguments must match:\npython code/fine_tune.py <data_dir/> <model_dir/>')
#         print('Example: python code/fine_tune.py data/dogs_cats/ model/dog_cats/')
#         sys.exit(2)
#     else:
#         data_dir = os.path.abspath(sys.argv[1])
#         train_dir = os.path.join(os.path.abspath(data_dir), 'train')  # Inside, each class should have it's own folder
#         validation_dir = os.path.join(os.path.abspath(data_dir), 'validation')  # each class should have it's own folder
#         model_dir = os.path.abspath(sys.argv[2])

#         os.makedirs(os.path.join(os.path.abspath(data_dir), 'preview'), exist_ok=True)
#         os.makedirs(model_dir, exist_ok=True)

#     train(train_dir, validation_dir, model_dir)  # train model

#     # release memory
#     k.clear_session()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

Epoch 1/2
7/7 [==============================] - 6s 801ms/step - loss: 1.1047 - acc: 0.8113 - val_loss: 0.0953 - val_acc: 0.9583

Epoch 00001: val_acc improved from -inf to 0.95833, saving model to C:\Users\COE1\top_model_weights.h5
Epoch 2/2
7/7 [==============================] - 3s 412ms/step - loss: 0.2983 - acc: 0.8879 - val_loss: 0.0680 - val_acc: 0.9688

Epoch 00002: val_acc improved from 0.95833 to 0.96875, saving model to C:\Users\COE1\top_model_weights.h5

Starting to Fine Tune Model

Epoch 1/10
7/7 [==============================] - 4s 564ms/step - loss: 0.2457 - acc: 0.9107 - val_loss: 0.0680 - val_acc: 0.9688

Epoch 00001: val_acc improved from -inf to 0.96875, saving model to C:\Users\COE1\model_weights.h5
Epoch 2/10
7/7 [==============================] - 3s 425ms/step - loss: 0.3575 - acc: 0.8809 - val_loss: 0.0680 - val_acc: 0.9688

Epoch 00002: val_acc did not improve
Epoch 3/10
7/7 [==============================] - 3s 445ms/step - loss: 0.0748 - acc: 0.9730 - val_loss